# Tox21 GINENet

We’re solving a multi-task binary toxicity prediction problem on Tox21. Concretely, for each molecule the model outputs 12 probabilities, one for each assay (e.g. NR-AR, SR-ARE, p53, etc.). At training time we use a binary cross-entropy loss (with masking for missing labels) over those 12 tasks, and at the end of each epoch we compute the ROC-AUC per task (then average) on the held-out validation set to see how well the model is distinguishing actives vs. inactives across all assays.

In [1]:
%pip -q install rdkit-pypi torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 53.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 59.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np

# Pytorch Imports
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Subset, WeightedRandomSampler
from torch.serialization import safe_globals, add_safe_globals

# Pytorch Geometric Imports
from torch_geometric.data import InMemoryDataset, Data
from torch_geometric.loader import DataLoader
from torch_geometric.data.data import DataTensorAttr, DataEdgeAttr
from torch_geometric.data.storage import GlobalStorage
from torch_geometric.nn import (
    GINEConv,
    GatedGraphConv,
    Set2Set,
    GlobalAttention,
    global_add_pool
)
from torch_geometric.nn.models import JumpingKnowledge

# RDKit Imports
from rdkit.Chem.Scaffolds import MurckoScaffold
from rdkit.Chem import MolFromSmiles, MolToSmiles, rdchem
from rdkit.Chem import Descriptors

from sklearn.metrics import roc_auc_score
import os

In [3]:
!wget wget https://raw.githubusercontent.com/deepchem/deepchem/master/datasets/tox21.csv.gz
!gunzip tox21.csv.gz
!mkdir -p raw
!mv tox21.csv raw/

--2025-05-27 22:22:43--  http://wget/
Resolving wget (wget)... failed: Name or service not known.
wget: unable to resolve host address ‘wget’
--2025-05-27 22:22:43--  https://raw.githubusercontent.com/deepchem/deepchem/master/datasets/tox21.csv.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 125310 (122K) [application/octet-stream]
Saving to: ‘tox21.csv.gz’

tox21.csv.gz        100%[===================>] 122.37K  --.-KB/s    in 0.02s   

2025-05-27 22:22:43 (5.73 MB/s) - ‘tox21.csv.gz’ saved [125310/125310]

FINISHED --2025-05-27 22:22:43--
Total wall clock time: 0.3s
Downloaded: 1 files, 122K in 0.02s (5.73 MB/s)


In [4]:
add_safe_globals([DataTensorAttr, DataEdgeAttr, GlobalStorage])

## Dataset

In [5]:
class Tox21Dataset(InMemoryDataset):
    def __init__(self, root: str = '.', transform=None, pre_transform=None):
        """
        Expects:
        root/
            raw/tox21.csv
        Will create:
            processed/data.pt
        """
        super().__init__(root, transform, pre_transform)
        # Load the processed data
        with safe_globals([DataTensorAttr, DataEdgeAttr, GlobalStorage]):
            self.data, self.slices = torch.load(self.processed_paths[0])

    @property
    def raw_file_names(self):
        # File expected in root/raw/
        return ['tox21.csv']

    @property
    def processed_file_names(self):
        return ['data.pt']

    def download(self):
        # No download step needed; CSV is already in place
        return

    def process(self):
        df = pd.read_csv(self.raw_paths[0])
        df = df.fillna(value=0)
        
        if "mol_id" in df.columns:
            df = df.drop(columns=["mol_id"])

        # 3) convert all non-smiles columns to numeric
        non_smiles = [c for c in df.columns if c != "smiles"]
        df[non_smiles] = df[non_smiles].apply(pd.to_numeric, errors="coerce")
        df = df.reset_index(drop=True)

        data_list = []
        bond_types = [
            rdchem.BondType.SINGLE,
            rdchem.BondType.DOUBLE,
            rdchem.BondType.TRIPLE,
            rdchem.BondType.AROMATIC,
        ]
        stereo_types = [
            rdchem.BondStereo.STEREONONE,
            rdchem.BondStereo.STEREOZ,
            rdchem.BondStereo.STEREOE,
            rdchem.BondStereo.STEREOANY
        ]

        for _, row in df.iterrows():
            smiles = row["smiles"]
            mol = MolFromSmiles(smiles, sanitize=True)
            if mol is None:
                continue

            # node features x
            atom_feats = [
                [
                    a.GetAtomicNum(),
                    a.GetDegree(),
                    a.GetFormalCharge(),
                    a.GetNumRadicalElectrons(),
                    a.GetTotalNumHs(),
                    int(a.GetIsAromatic()),
                    int(a.IsInRing())
                ]
                for a in mol.GetAtoms()
            ]
            x = torch.tensor(atom_feats, dtype=torch.float32)

            # get molecule descriptors
            descriptors = [
                Descriptors.MolWt(mol),
                Descriptors.MolLogP(mol),
                Descriptors.TPSA(mol),
                Descriptors.NumHAcceptors(mol),
                Descriptors.NumHDonors(mol)
            ]

            # build edge_index AND edge_attr in lock‐step
            edge_index = []
            edge_attr  = []
            for bond in mol.GetBonds():
                i, j = bond.GetBeginAtomIdx(), bond.GetEndAtomIdx()
                # one‐hot encode this bond’s type
                bt = bond.GetBondType()
                bfeat = [int(bt == t) for t in bond_types]
                st = bond.GetStereo()
                sfeat = [int(st == s) for s in stereo_types]
                feat = bfeat + sfeat

                # add both directions
                edge_index += [[i, j], [j, i]]
                edge_attr  += [feat, feat]

            edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()
            edge_attr  = torch.tensor(edge_attr, dtype=torch.float32)  # [2E, len(bond_types)]
            desc = torch.tensor(descriptors, dtype=torch.float32)

            # your labels
            y = torch.tensor(row[[c for c in df.columns if c!="smiles"]].tolist(), dtype=torch.float32)

            
            data_list.append(Data(
                x=x,
                edge_index=edge_index,
                edge_attr=edge_attr,
                y=y,
                desc=desc
            ))

        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])


In [6]:
dataset = Tox21Dataset('.')

Processing...
[22:22:44] WARNING: not removing hydrogen atom without neighbors
Done!


A **Murcko Scaffold** is a technique used to group molecules based on their core structural components. It identifies the essential building blocks by removing side chains and other non-core components, leaving behind the important ring systems and connecting chains. This method is widely used in medicinal chemistry and drug design to identify core structures that have preferential activity against specific targets, which is very useful in **molecular property prediction**.

In [7]:
def GetMurckoScaffold(data, train_frac: float, val_frac: float, test_frac: float):
    scaffold_indices = {}
    for idx, smiles in enumerate(data):
        scaffold_smiles = MurckoScaffold.MurckoScaffoldSmiles(smiles)
        #scaffold_smiles = MolToSmiles(scaffold)
        scaffold_indices.setdefault(scaffold_smiles, []).append(idx)

    groups = sorted(scaffold_indices.values(), key=len, reverse=True)
    n_total = len(data)
    n_train = int(train_frac * n_total)
    n_valid = int(val_frac * n_total)

    train_idx, valid_idx, test_idx = [], [], []
    for group in groups:
        if len(train_idx) + len(group) <= n_train:
            train_idx.extend(group)
        elif len(valid_idx) + len(group) <= n_valid:
            valid_idx.extend(group)
        else:
            test_idx.extend(group)

    return train_idx, valid_idx, test_idx

## GNN Model

In [8]:
np.random.seed(1638)
torch.manual_seed(1638)

In [28]:
class GINEModelWithDesc(nn.Module):
    def __init__(
        self,
        input_dim: int,
        hidden_dim: int,
        desc_dim: int,
        num_tasks: int,
        n_layers: int,
        set2set_steps: int = 3,
    ):
        super().__init__()
        
        # First GINE layer
        self.conv1 = GINEConv(
            nn.Sequential(
                nn.Linear(input_dim, hidden_dim),
                nn.BatchNorm1d(hidden_dim),
                nn.ReLU(),
                nn.Linear(hidden_dim, hidden_dim),
            ),
            edge_dim=8,
        )

        # Additional GINE layers with skip connections
        self.conv_block = nn.ModuleList()
        for _ in range(n_layers):
            mlp = nn.Sequential(
                nn.Linear(hidden_dim, hidden_dim),
                nn.BatchNorm1d(hidden_dim),
                nn.ReLU(),
                nn.Linear(hidden_dim, hidden_dim),
            )
            self.conv_block.append(GINEConv(mlp, edge_dim=8))

        # Optional gated updates
        self.gate = GatedGraphConv(hidden_dim, 3)
        self.bn_gate = nn.BatchNorm1d(hidden_dim)

        # Replace mean pool with Set2Set
        self.pool = Set2Set(hidden_dim, processing_steps=set2set_steps)

        # Fully connected: include descriptor dimension
        self.fc = nn.Linear(2 * hidden_dim + desc_dim, num_tasks)

    def forward(self, x, edge_index, edge_attr, batch, descriptors):
        # 1st GINE layer
        x = self.conv1(x, edge_index, edge_attr)

        # GINE layers + skip connections + dropout
        for block in self.conv_block:
            h = block(x, edge_index, edge_attr)
            x = (x + h).relu()
            x = F.dropout(x, p=0.1, training=self.training)
    
        # (Optional) gated graph conv and BN
        x = self.gate(x, edge_index)
        x = self.bn_gate(x)

        # Set2Set pooling: output shape [batch_size, 2*hidden_dim]
        x = self.pool(x, batch)

        # Concatenate descriptor features
        # descriptors should be shape [batch_size, desc_dim]
        descs = descriptors.view(-1, 5)
        x = torch.cat([x, descs], dim=1)

        # Final prediction
        return self.fc(x)

## Train the Model

In [46]:
def train_epoch(model, loader, optimizer, criterion, device, use_desc=True):
    model.train()
    total_loss = 0
    for batch in loader:
        batch = batch.to(device)
        optimizer.zero_grad()
        if use_desc:
            logits = model(batch.x, batch.edge_index, batch.edge_attr, batch.batch, batch.desc)
        else:
            logits = model(batch.x, batch.edge_index, batch.edge_attr, batch.batch)
        mask = (batch.y >= 0).float()
        bs, nt = logits.size()
        batch_y = batch.y.view(bs, nt)
        mask = mask.view(bs, nt)
        loss = (criterion(logits, batch_y) * mask).sum() / mask.sum()
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * batch.num_graphs
    return total_loss / len(loader.dataset)

In [47]:
def evaluate(model, loader, device, use_desc=True):
    model.eval()
    y, preds = [], []
    with torch.no_grad():
        for batch in loader:
            batch = batch.to(device)
            if use_desc:
                logits = model(batch.x, batch.edge_index, batch.edge_attr, batch.batch, batch.desc)
            else:
                logits = model(batch.x, batch.edge_index, batch.edge_attr, batch.batch)
            y.append(batch.y.cpu())
            preds.append(torch.sigmoid(logits).cpu())
    return torch.cat(preds, dim=0).numpy(), torch.cat(y, dim=0).numpy()

## Main

In [31]:
df = pd.read_csv('raw/tox21.csv').fillna(0).reset_index(drop=True)
smiles_list = df['smiles'].tolist()
train_idx, valid_idx, test_idx = GetMurckoScaffold(smiles_list, 0.8, 0.1, 0.1)

train_ds = Subset(dataset, train_idx)
val_ds   = Subset(dataset, valid_idx)
test_ds  = Subset(dataset, test_idx)

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)#, sampler=sampler)
val_loader   = DataLoader(val_ds,   batch_size=32, shuffle=True)
test_loader  = DataLoader(test_ds,  batch_size=32)

sample = dataset[0]
in_channels = sample.x.size(1)
num_tasks   = sample.y.size(0)
desc_dim = sample.desc.size(0)

print("There are", num_tasks, "tasks.")

[22:44:00] WARNING: not removing hydrogen atom without neighbors


There are 12 tasks.


In [32]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GINEModelWithDesc(
    in_channels,
    hidden_dim = 384,
    num_tasks = num_tasks,
    desc_dim = desc_dim,
    n_layers = 3
).to(device)

print(model)

GINEModelWithDesc(
  (conv1): GINEConv(nn=Sequential(
    (0): Linear(in_features=7, out_features=384, bias=True)
    (1): BatchNorm1d(384, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Linear(in_features=384, out_features=384, bias=True)
  ))
  (conv_block): ModuleList(
    (0-2): 3 x GINEConv(nn=Sequential(
      (0): Linear(in_features=384, out_features=384, bias=True)
      (1): BatchNorm1d(384, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): Linear(in_features=384, out_features=384, bias=True)
    ))
  )
  (gate): GatedGraphConv(384, num_layers=3)
  (bn_gate): BatchNorm1d(384, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): Set2Set(384, 768)
  (fc): Linear(in_features=773, out_features=12, bias=True)
)


In [33]:
optimizer = optim.AdamW(
    model.parameters(),
    lr=1e-4,
    weight_decay=1e-5,
)

scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,
    mode="max",           # we track val AUC, so “max”
    factor=0.5,           # halve the lr
    patience=5,           # after 5 epochs with no AUC gain
    min_lr=1e-6
)

In [34]:
class FocalLoss(nn.Module):
    def __init__(self, alpha=0.25, gamma=2.0, reduction='mean'):
        super().__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, logits, targets):
        # logits: shape [N, ...], raw outputs
        # targets: same shape, 0 or 1
        probas = torch.sigmoid(logits)
        # p_t: prob of true class
        p_t = probas * targets + (1 - probas) * (1 - targets)
        # alpha factor
        alpha_factor = self.alpha * targets + (1 - self.alpha) * (1 - targets)
        # focal weight
        focal_weight = alpha_factor * (1 - p_t) ** self.gamma
        # binary cross‐entropy per example
        bce = F.binary_cross_entropy_with_logits(logits, targets, reduction='none')
        loss = focal_weight * bce

        if self.reduction == 'mean':
            return loss.mean()
        elif self.reduction == 'sum':
            return loss.sum()
        else:
            return loss

criterion = FocalLoss(alpha=0.10, gamma=1.0, reduction='none')

In [35]:
for epoch in range(1, 21):
    loss = train_epoch(model, train_loader, optimizer, criterion, device)
    ps_val, ys_val = evaluate(model, val_loader, device)
    ys_val = ys_val.reshape(ps_val.shape)
    aucs = []
    for i in range(ps_val.shape[1]):
        mask = ys_val[:, i] >= 0
        if mask.sum() > 0:
            aucs.append(roc_auc_score(ys_val[mask, i], ps_val[mask, i]))
    scheduler.step(np.mean(aucs)) # we use the mean like below
    print(f"Epoch {epoch:02d} | Loss: {loss:.4f} | Val AUC: {np.mean(aucs):.3f}")

Epoch 01 | Loss: 0.2141 | Val AUC: 0.532
Epoch 02 | Loss: 0.0377 | Val AUC: 0.549
Epoch 03 | Loss: 0.0300 | Val AUC: 0.623
Epoch 04 | Loss: 0.0249 | Val AUC: 0.641
Epoch 05 | Loss: 0.0220 | Val AUC: 0.652
Epoch 06 | Loss: 0.0206 | Val AUC: 0.648
Epoch 07 | Loss: 0.0197 | Val AUC: 0.655
Epoch 08 | Loss: 0.0189 | Val AUC: 0.648
Epoch 09 | Loss: 0.0180 | Val AUC: 0.668
Epoch 10 | Loss: 0.0174 | Val AUC: 0.668
Epoch 11 | Loss: 0.0172 | Val AUC: 0.682
Epoch 12 | Loss: 0.0169 | Val AUC: 0.662
Epoch 13 | Loss: 0.0187 | Val AUC: 0.632
Epoch 14 | Loss: 0.0174 | Val AUC: 0.667
Epoch 15 | Loss: 0.0169 | Val AUC: 0.677
Epoch 16 | Loss: 0.0167 | Val AUC: 0.679
Epoch 17 | Loss: 0.0166 | Val AUC: 0.684
Epoch 18 | Loss: 0.0164 | Val AUC: 0.677
Epoch 19 | Loss: 0.0163 | Val AUC: 0.703
Epoch 20 | Loss: 0.0162 | Val AUC: 0.684


In [37]:
ps_test, ys_test = evaluate(model, test_loader, device)
ys_test = ys_test.reshape(ps_test.shape)
aucs = []
for i in range(ps_test.shape[1]):
    mask = ys_test[:, i] >= 0
    if mask.sum() > 0:
        aucs.append(roc_auc_score(ys_test[mask, i], ps_test[mask, i]))
print(f"Testing | Loss: {loss:.4f} | Test AUC: {np.mean(aucs):.3f}")

Testing | Loss: 0.0162 | Test AUC: 0.700


### GINE with Jumping Knowledge Model

In [38]:
class GINEWithJK(nn.Module):
    def __init__(
        self,
        input_dim: int,
        hidden_dim: int,
        num_tasks: int,
        n_layers: int,
        jk_mode: str = 'cat', 
        set2set_steps: int = 3,
    ):
        super().__init__()
        self.n_layers = n_layers

        # first GINE
        self.conv1 = GINEConv(
            nn.Sequential(
                nn.Linear(input_dim, hidden_dim),
                nn.BatchNorm1d(hidden_dim),
                nn.ReLU(),
                nn.Linear(hidden_dim, hidden_dim),
            ),
            edge_dim=8,
        )

        # additional GINE layers
        self.convs = nn.ModuleList()
        for _ in range(n_layers):
            mlp = nn.Sequential(
                nn.Linear(hidden_dim, hidden_dim),
                nn.BatchNorm1d(hidden_dim),
                nn.ReLU(),
                nn.Linear(hidden_dim, hidden_dim),
            )
            self.convs.append(GINEConv(mlp, edge_dim=8))

        # Jumping Knowledge combiner
        # if mode='cat', output size = hidden_dim*(n_layers+1)
        self.jk = JumpingKnowledge(mode=jk_mode)

        # pooling: either Set2Set or mean
        # If using Set2Set, pool_in_dim = hidden_dim*(n_layers+1)
        self.pool = Set2Set(hidden_dim * (n_layers + 1), processing_steps=set2set_steps)
        # otherwise: self.pool = global_mean_pool

        # final FC — input dim doubles due to Set2Set concat
        self.fc = nn.Linear(2 * hidden_dim * (n_layers + 1), num_tasks)

    def forward(self, x, edge_index, edge_attr, batch):
        xs = []
        # layer 0
        x = F.relu(self.conv1(x, edge_index, edge_attr))
        xs.append(x)

        # layers 1…n
        for conv in self.convs:
            x = F.relu(conv(x, edge_index, edge_attr))
            xs.append(x)

        # combine via JK
        x = self.jk(xs)   # if 'cat', shape = [total_nodes, hidden_dim*(n_layers+1)]

        # pooling
        x = self.pool(x, batch)  # -> [batch_size, 2*hidden_dim*(n_layers+1)]

        # final prediction
        return self.fc(x)

In [39]:
jk_model = GINEWithJK(
    in_channels,
    hidden_dim = 384,
    num_tasks = num_tasks,
    n_layers = 3,
    jk_mode = 'cat',
    set2set_steps = 3,
).to(device)

print(jk_model)

GINEWithJK(
  (conv1): GINEConv(nn=Sequential(
    (0): Linear(in_features=7, out_features=384, bias=True)
    (1): BatchNorm1d(384, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Linear(in_features=384, out_features=384, bias=True)
  ))
  (convs): ModuleList(
    (0-2): 3 x GINEConv(nn=Sequential(
      (0): Linear(in_features=384, out_features=384, bias=True)
      (1): BatchNorm1d(384, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): Linear(in_features=384, out_features=384, bias=True)
    ))
  )
  (jk): JumpingKnowledge(cat)
  (pool): Set2Set(1536, 3072)
  (fc): Linear(in_features=3072, out_features=12, bias=True)
)


In [40]:
jk_optimizer = optim.AdamW(
    jk_model.parameters(),
    lr=1e-4,
    weight_decay=1e-5,
)

jk_scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    jk_optimizer,
    mode="max",           # we track val AUC, so “max”
    factor=0.5,           # halve the lr
    patience=5,           # after 5 epochs with no AUC gain
    min_lr=1e-6
)

In [41]:
jk_criterion = FocalLoss(alpha=0.2, gamma=1.4, reduction='none')

In [49]:
for epoch in range(1, 21):
    loss = train_epoch(jk_model, train_loader, jk_optimizer, jk_criterion, device, False)
    ps_val, ys_val = evaluate(jk_model, val_loader, device, False)
    ys_val = ys_val.reshape(ps_val.shape)
    aucs = []
    for i in range(ps_val.shape[1]):
        mask = ys_val[:, i] >= 0
        if mask.sum() > 0:
            aucs.append(roc_auc_score(ys_val[mask, i], ps_val[mask, i]))
    jk_scheduler.step(np.mean(aucs)) # we use the mean like below
    print(f"Epoch {epoch:02d} | Loss: {loss:.4f} | Val AUC: {np.mean(aucs):.3f}")

Epoch 01 | Loss: 0.0228 | Val AUC: 0.687
Epoch 02 | Loss: 0.0220 | Val AUC: 0.697
Epoch 03 | Loss: 0.0214 | Val AUC: 0.704
Epoch 04 | Loss: 0.0210 | Val AUC: 0.706
Epoch 05 | Loss: 0.0206 | Val AUC: 0.714
Epoch 06 | Loss: 0.0203 | Val AUC: 0.713
Epoch 07 | Loss: 0.0200 | Val AUC: 0.723
Epoch 08 | Loss: 0.0196 | Val AUC: 0.715
Epoch 09 | Loss: 0.0193 | Val AUC: 0.700
Epoch 10 | Loss: 0.0192 | Val AUC: 0.716
Epoch 11 | Loss: 0.0189 | Val AUC: 0.711
Epoch 12 | Loss: 0.0187 | Val AUC: 0.722
Epoch 13 | Loss: 0.0185 | Val AUC: 0.717
Epoch 14 | Loss: 0.0176 | Val AUC: 0.722
Epoch 15 | Loss: 0.0172 | Val AUC: 0.718
Epoch 16 | Loss: 0.0171 | Val AUC: 0.717
Epoch 17 | Loss: 0.0169 | Val AUC: 0.714
Epoch 18 | Loss: 0.0168 | Val AUC: 0.724
Epoch 19 | Loss: 0.0165 | Val AUC: 0.716
Epoch 20 | Loss: 0.0164 | Val AUC: 0.727


In [51]:
ps_test, ys_test = evaluate(jk_model, test_loader, device, False)
ys_test = ys_test.reshape(ps_test.shape)
aucs = []
for i in range(ps_test.shape[1]):
    mask = ys_test[:, i] >= 0
    if mask.sum() > 0:
        aucs.append(roc_auc_score(ys_test[mask, i], ps_test[mask, i]))
print(f"Testing | Loss: {loss:.4f} | Test AUC: {np.mean(aucs):.3f}")

Testing | Loss: 0.0164 | Test AUC: 0.718
